# Install dependencies

In [17]:
# !pip install -r requirements.txt

# Import libraries

In [18]:
import os
from dotenv import find_dotenv, load_dotenv

from langchain.agents import create_openai_tools_agent
from langchain.agents.agent import AgentExecutor
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain_core.messages import AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import AzureChatOpenAI
from langchain.sql_database import SQLDatabase

# Load environment varaible

In [19]:
_ = load_dotenv(find_dotenv(filename=".env", raise_error_if_not_found=True))

In [20]:
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("api_base")

In [21]:
# LangSmith (A unified platform for debugging, testing, evaluating and monitoring LLM applications)
# ⚠️This is just for testing, it is not recommended to use it. Data will go outside your network
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

# Setup database

In [23]:
db = SQLDatabase.from_uri(os.getenv("PG_URI"))

# Setup LLM

In [24]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4",
    openai_api_version=os.getenv("api_version"),
    azure_endpoint=os.getenv("api_base"),
    temperature=0,
    max_tokens=4000,
    max_retries=20,
)

# Setup Agent

In [25]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()
tools = toolkit.get_tools()

In [26]:
messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=SQL_FUNCTIONS_SUFFIX),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)
prompt = prompt.partial(**context)

In [27]:
agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

# Ask a query

In [28]:
agent_executor.invoke({"input": "how many traces are there in the database?"})



> Entering new AgentExecutor chain...



Invoking: `sql_db_list_tables` with ``


Account, Example, Session, _prisma_migrations, api_keys, cron_jobs, dataset_items, dataset_run_items, dataset_runs, datasets, events, memberships, observations, pricings, projects, scores, traces, users, verification_tokens
Invoking: `sql_db_schema` with `{'table_names': 'traces'}`
responded: The table that seems to contain the traces is called "traces". Let's check its schema to understand its structure.


CREATE TABLE traces (
	id TEXT NOT NULL, 
	timestamp TIMESTAMP(3) WITHOUT TIME ZONE DEFAULT CURRENT_TIMESTAMP NOT NULL, 
	name TEXT, 
	project_id TEXT NOT NULL, 
	metadata JSONB, 
	external_id TEXT, 
	user_id TEXT, 
	release TEXT, 
	version TEXT, 
	public BOOLEAN DEFAULT false NOT NULL, 
	CONSTRAINT traces_pkey PRIMARY KEY (id), 
	CONSTRAINT traces_project_id_fkey FOREIGN KEY(project_id) REFERENCES projects (id) ON DELETE CASCADE ON UPDATE CASCADE
)

/*
3 rows from traces table:
id	timestamp	name	project_id	metadata	external_id	user_id	relea

{'input': 'how many traces are there in the database?',
 'output': 'There are 663 traces in the database.'}